   1  : 1.58,
   0.1: 2.71,
  0.05: 3.84, 
  0.01: 6.63,
 0.005: 7.88, 
 0.001: 10.83

In [18]:
# 2.71 - 1.13

1.58

In [1]:
import pandas as pd
import numpy as np
from joblib import dump

from chi import square as cs

In [2]:
data = pd.read_excel("komentar/data_clean_goejk.xlsx")
komentar = data.komentar.tolist()
label = np.array(data.label.tolist())
y = label
data.head()

,Unnamed: 0,label,komentar,komentar_lama
0,0,0,min mau tanya apa go tix batas waktu bayar mal...,"min saya mau tanya,apa go tix tidak ada batas..."
1,1,0,ngapain kamu naik motor 2an suami plot twist s...,"""Ngapain kamu naik motor berdua2an suami saya?..."
2,2,0,nge blank i think y,"""Nge-blank"", i think? ðŸ˜…"
3,3,0,ooh kirabmenteng deket bxc,"""ooh saya kirabmenteng yang deket bxc"""
4,4,0,siap teman makan siang mu tak perlu anjak meja...,#asinansayurdanrujakjuhi siap temani makan sia...


# Ekstraksi dan Seleksi Fitur

In [3]:
sf = cs.ChiSquare(alpha = 0.1)
sf.find_best_features(komentar, label, fitur = None)

In [4]:
print("jumlah fitur terbaik:",len(sf.best_features),"dari",len(sf.fitur), "fitur")
best_features = sf.best_features

jumlah fitur terbaik: 1492 dari 6351 fitur


# Pemodelan

## Pemodelan TF-IDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(vocabulary = best_features)#
X = vectorizer.fit_transform(komentar)
save = vectorizer.fit(komentar)
dump(save, "model/tfidf.w")
# print(vectorizer.get_feature_names())
fitur = vectorizer.get_feature_names()
print(X.shape)

(3210, 1492)


## Pemodelan SVM

In [6]:
from sklearn.svm import SVC
clf = SVC()#gamma='auto'

# Pemodelan dan Validasi Model

In [13]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

start = 1
stop = 2
step = 0.1
alpha_list = np.arange(start,stop+step,step)
print(alpha_list)
akurasi = list()

ix = 0 #just counter
for alpha in alpha_list:
    clf.set_params(gamma=alpha)
    temp_akurasi = list()
    for train_index, test_index in skf.split(X, y):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        ##Trainin Model menggunakan sebagian data latih (K-fold)
        clf.fit(X_train,y_train)
        
        #prediksi menggunakan data uji dari k-fold
        y_pred = clf.predict(X_test)
        temp_akurasi.append(accuracy_score(y_test, y_pred))
    rerata_akurasi = sum(temp_akurasi)/len(temp_akurasi)
    akurasi.append(rerata_akurasi)
    ix +=1
    if ix%100 !=0:
        print(".", end="")
    else:
#         print(alpha, ' | ', rerata_akurasi)
        print("")

        pass

ditt = {
    "gamma":alpha_list,
    "akurasi":akurasi
}
index = akurasi.index(max(akurasi))
acc = pd.DataFrame.from_dict(ditt)
acc

[1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2. ]
...........

,gamma,akurasi
0,1.0,0.849533
1,1.1,0.851402
2,1.2,0.854517
3,1.3,0.855140
4,1.4,0.856698
5,1.5,0.855140
6,1.6,0.853583
7,1.7,0.853271
8,1.8,0.852025
9,1.9,0.849844


In [16]:
print(alpha_list[index])
print("Akurasi", akurasi[index])

1.4000000000000004
Akurasi 0.8566978193146417


In [15]:
# print(alpha_list[index])
# print(akurasi[index])

# Menyimpan Model

In [17]:
clf.set_params(gamma=alpha_list[index])

#Trainin Model menggunakan seluruh data latih
clf.fit(X, y)
dump(clf, "model/svm.m")

['model/svm.m']